# cadCAD x TrojanDAO

In parts [1](../robot-marbles-part-1/robot-marbles-part-1.ipynb) and [2](../robot-marbles-part-2/robot-marbles-part-2.ipynb) we introduced the 'language' in which a system must be described in order for it to be interpretable by cadCAD and some of the basic concepts of the library:
* State Variables
* Timestep
* State Update Functions
* Partial State Update Blocks
* Simulation Configuration Parameters
* Policies

So far, all the examples referred to deterministic systems: no matter how many times you ran one of those simulations, the results would be the same. However, systems are more commonly non-deterministic, and modelling them as deterministic might be an oversimplification sometimes. 


In [8]:
import numpy as np
import random as rd
# a = np.array([]) 
# x_user_balances = np.array([120,120,120,120,120,120,120])
# actusr = x_user_balances[rd.randint(1,7)]


3

In [19]:
%%capture
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
# Assumption: we model interactions on the BC and DAO pools, treating user interactions stochastically

initial_conditions = {
  'BC_reserve': 0,
  'token_holders': np.array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]),
  'n': 8, # max number of token holders including guildbank and DAO pool
  'DAO_pool_index': 0,
  'guildbank_index': 1,
  'amount_to_mint': 100, # in eth
  'amount_to_burn': 0.2, # proportion of holdings
  'price': 1, # in eth/tok
  'total_tokens': 0,
  'action': 'mint',
  'update_index': 0,
  'DAO_tax_rate': 0.02,
  'redist_tax_rate': 0.01,
  'burn_tax_rate': 0.03,
  'redist': 0,
}

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
# In this example, we'll run the simulation once (N=1) and its duration will be of 10 timesteps
# We'll cover the `M` key in a future article. For now, let's leave it empty
simulation_parameters = {
    'T': range(100),
    'N': 1,
    'M': {}
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Investor,liquidator logic in a Policy Function
def choose_holder(params, step, sL, s):
    index = np.random.randint(2, s['n'])
    print("Index to act on is %s" % (index))
    return ({'update_index': index})

# def investor(params, step, sL, s):
#     return()

# def pers_burn(params, step, sL, s):
#     return()

# def DAOburn(params, step, sL, s):
#     return()

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# State update functions
def choose_action(params, step, sL, s):
    p = np.random.randint(0, 100)
    return ({'action': 'mint'})

def update_BC_reserve_mint(params, step, sL, s, _input):
    y = 'BC_reserve'
    amount_to_mint = 0
    if (_input['update_index'] > 1):
        amount_to_mint = s['amount_to_mint']
    x = s['BC_reserve'] + amount_to_mint # newly minted amount
    print("New BC reserve is %s" % (x))
    return (y, x)

def update_BC_reserve_burn(params, step, sL, s, _input):
    y = 'BC_reserve'
    update_index = _input['update_index']
    amount_to_burn = s['amount_to_burn'] * s['token_holders'][update_index]
    x = s['BC_reserve'] - amount_to_burn * (1 - s['burn_tax_rate'])
    return (y, x)

def update_total_tokens_mint(params, step, sL, s, _input):
    y = 'total_tokens'
    amount_to_mint = s['amount_to_mint']
    x = s['total_tokens'] + amount_to_mint
    return (y, x)

def update_total_tokens_burn(params, step, sL, s, _input):
    y = 'total_tokens'
    token_holders = s['token_holders']
    update_index = _input['update_index']
    amount_to_burn = s['amount_to_burn'] * token_holders[update_index]
    x = s['total_tokens'] - amount_to_burn
    return (y, x)

def update_token_holders_mint(params, step, sL, s, _input):
    y = 'token_holders'
    x = s['token_holders']
    update_index = _input['update_index']
    amount_to_mint = s['amount_to_mint'] * (1 - s['DAO_tax_rate'] - s['redist_tax_rate'])
    x[update_index] = x[update_index] + amount_to_mint
    DAO_tax_amount = s['amount_to_mint'] * s['DAO_tax_rate']
    x[0] = x[0] + DAO_tax_amount
    print(x)
    return (y, x)

def update_token_holders_burn(params, step, sL, s, _input):
    y = 'token_holders'
    x = s['token_holders']
    update_index = _input['update_index']
    amount_to_burn = s['amount_to_burn'] * x[update_index]
    DAO_tax_amount = amount_to_burn * s['burn_tax_rate']
    x[update_index] = x[update_index] - amount_to_burn
    x[0] = x[0] + DAO_tax_amount
    print(x)
    return (y, x)

def update_redistribution_amount_mint(params, step, sL, s, _input):
    y = 'redist'
    update_index = _input['update_index']
    redist_tax_amount = 0
    if (update_index > 1):
        redist_tax_amount = s['amount_to_mint'] * s['redist_tax_rate']
    print(redist_tax_amount)
    return (y, redist_tax_amount)

def update_redistribution_amount_burn(params, step, sL, s, _input):
    y = 'redist'
    redist_tax_amount = 0
    return (y, redist_tax_amount)

def redistribute(params, step, sL, s, _input):
    y = 'token_holders'
    x = s['token_holders']
    n = s['n']
    redist = s['redist']
    total_tokens = s['total_tokens']
    print("Redistribute %s based on total %s" % (redist, total_tokens))
    for i in range(0, n):
        x[i] = x[i] + redist * x[i] / total_tokens
    print(x)
    return (y, x)

# def upd8_DAOpool(params, step, sL, s, _input):
#     y = 1
#     x = 1
#     return(y,x)

# def upd8_BCreserve(params, step, sL, s, _input):
#     y = 1
#     x = 1
#     return(y,x)

# def upd8_M_circulating(params, step, sL, s, _input):
#     y = 1
#     x = 1
#     return(y,x)

# def upd8_x_userbalances(params, step, sL, s, _input):
#     y = 1
#     x = 1
#     return(y,x)

# def activeuser(params, step, sL, s, _input):
#     actusr = rd.randint(1,7)
#     return(actusr)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, 
# the user specifies if state update functions will be run in series or in parallel
# and the policy functions that will be evaluated in that block

# Multiple blocks: in series
# All in one block: in parallel

partial_state_update_blocks = [
    { 
        'policies': {
            'choose_holder': choose_holder, 
        },
        'variables': {
            'BC_reserve': update_BC_reserve_burn,
            'token_holders': update_token_holders_burn,
            'redist': update_redistribution_amount_burn,
            'total_tokens': update_total_tokens_burn,
        }
    }
]

# partial_state_update_blocks = [
#     { 
#         'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
# #             '': ,
# #             '':
#         },
#         'variables': { # The following state variables will be updated simultaneously
#             'DAO_pool': upd8_DAOpool,
#             'BC_reserve': upd8_BCreserve
#         }
#     }
# ]


In [20]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
from cadCAD.configuration import Configuration
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )

from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results

%matplotlib inline
import pandas as pd
df = pd.DataFrame(raw_result)


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: single_proc: [<cadCAD.configuration.Configuration object at 0x000002828803F948>]
Configurations: [<cadCAD.configuration.Configuration object at 0x000002828803F948>]
Index to act on is 3
[1006 1000 1000  800 1000 1000 1000 1000]
Index to act on is 2
[1012 1000  800  800 1000 1000 1000 1000]
Index to act on is 4
[1018 1000  800  800  800 1000 1000 1000]
Index to act on is 2
[1022 1000  640  800  800 1000 1000 1000]
Index to act on is 4
[1026 1000  640  800  640 1000 1000 1000]
Index to act on is 7
[1032 1000  640  800  640 1000 1000  800]
Index to act on is 7
[1036 1000  640  800  640 1000 1000  640]
Index to act on is 4
[1039 1000  640  800  512 1000 1000  640]
Index to act on is 4
[1042 1000  640  800  409

In [21]:
df.plot('timestep', ['BC_reserve', 'DAO_pool'], grid=True, 
        xticks=list(df['timestep'].drop_duplicates()), 
        colormap = 'RdYlGn',
        yticks=list(range(1+(df['BC_reserve']+df['DAO_pool']).max())));

KeyError: 'DAO_pool'

In [ ]:
# Plot values of iteration
df.set_index(['run', 'timestep', 'substep'])


# Non-determinism
Non-deterministic systems exhibit different behaviors on different runs for the same input. The order of heads and tails in a series of 3 coin tosses, for example, is non deterministic. 

Our robots and marbles system is currently modelled as a deterministic system. Meaning that every time we run the simulation: none of the robots act on timestep 1; robot 1 acts on timestep 2; robot 2 acts on timestep 3; an so on. 

If however we were to define that at every timestep each robot would act with a probability P, then we would have a non-deterministic (probabilistic) system. Let's make the following changes to our system.
* Robot 1: instead of acting once every two timesteps, there's a 50% chance it will act in any given timestep
* Robot 2: instead of acting once every three timesteps, there's a 33.33% chance it will act in any given timestep

In [ ]:
%%capture
from numpy.random import rand

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# We specify each of the robots logic in a Policy Function
robots_probabilities = [0.5,1/3] # Robot 1 acts with a 50% probability; Robot 2, 33.33%

def robot_arm_1(params, step, sL, s):
    _robotId = 1
    if rand()<robots_probabilities[_robotId-1]: # draw a random number between 0 and 1; if it's smaller than the robot's parameter, it acts
        return robot_arm(params, step, sL, s)
    else:
        return({'add_to_A': 0, 'add_to_B': 0}) # otherwise, the robot doesn't interfere with the system

def robot_arm_2(params, step, sL, s):
    _robotId = 2
    if rand()<robots_probabilities[_robotId-1]: # draw a random number between 0 and 1; if it's smaller than the robot's parameter, it acts
        return robot_arm(params, step, sL, s)
    else:
        return({'add_to_A': 0, 'add_to_B': 0}) # otherwise, the robot doesn't interfere with the system


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, 
# the user specifies if state update functions will be run in series or in parallel
# and the policy functions that will be evaluated in that block
partial_state_update_blocks = [
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'robot_arm_1': robot_arm_1,
            'robot_arm_2': robot_arm_2
        },
        'variables': { # The following state variables will be updated simultaneously
            'box_A': increment_A,
            'box_B': increment_B
        }
    }
]
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

If we run the simulation with those configurations, the system is unlikely to exhibit the same behavior as it did in its deterministic version

In [ ]:
%%capture
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results

df = pd.DataFrame(raw_result)

In [ ]:
df.plot('timestep', ['box_A', 'box_B'], grid=True, 
        xticks=list(df['timestep'].drop_duplicates()), 
        colormap = 'RdYlGn',
        yticks=list(range(1+(df['box_A']+df['box_B']).max())));

And if we run it again, it returns yet another result

In [ ]:
%%capture
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results

df = pd.DataFrame(raw_result)

In [ ]:
df.plot('timestep', ['box_A', 'box_B'], grid=True, 
        xticks=list(df['timestep'].drop_duplicates()), 
        colormap = 'RdYlGn',
        yticks=list(range(1+(df['box_A']+df['box_B']).max())));

In order to take advantage of cadCAD's Monte Carlo simulation features, we should modify the configuration file so as to define the number of times we want the same simulation to be run. This is done in the `N` key of the `simulation_parameters` dict.

In [ ]:
%%capture
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
# In this example, we'll run the simulation once (N=1) and its duration will be of 10 timesteps
# We'll cover the `M` key in a future article. For now, let's leave it empty
simulation_parameters = {
    'T': range(10),
    'N': 50, # We'll run the same simulation 50 times; the random events in each simulation are independent
    'M': {}
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results

df = pd.DataFrame(raw_result)

In [ ]:
from IPython.display import display
tmp_rows = pd.options.display.max_rows
pd.options.display.max_rows = 10
display(df.set_index(['run', 'timestep', 'substep']))
pd.options.display.max_rows = tmp_rows

Plotting two of those runs allows us to see the different behaviors over time.

In [ ]:
df[df['run']==1].plot('timestep', ['box_A', 'box_B'], grid=True,
                          xticks=list(df['timestep'].drop_duplicates()), 
                          yticks=list(range(11)),
                          colormap = 'RdYlGn');
df[df['run']==9].plot('timestep', ['box_A', 'box_B'], grid=True,
                            xticks=list(df['timestep'].drop_duplicates()), 
                            yticks=list(range(11)),
                            colormap = 'RdYlGn');

If we plot all those runs onto a single chart, we can see every possible trajectory for the number of marbles in each box.

In [ ]:
ax = None
for i in range(simulation_parameters['N']):
    ax = df[df['run']==i+1].plot('timestep', ['box_A', 'box_B'],
                                     grid=True,
                                     xticks=list(df['timestep'].drop_duplicates()), 
                                     yticks=list(range(1+max(df['box_A'].max(),df['box_B'].max()))),
                                     legend = (ax == None),
                                     colormap = 'RdYlGn',
                                     ax = ax
                                  )

For some analyses, it might make sense to look at the data in aggregate. Take the median for example:

In [ ]:
dfmc_median = df.groupby(['timestep', 'substep']).median().reset_index()
dfmc_median.plot('timestep', ['box_A', 'box_B'], 
                 grid=True,
                 xticks=list(dfmc_median['timestep'].drop_duplicates()), 
                 yticks=list(range(int(1+max(dfmc_median['box_A'].max(),dfmc_median['box_B'].max())))),
                 colormap = 'RdYlGn'
                )

Or look at edge cases

In [ ]:
max_final_A = df[df['timestep']==df['timestep'].max()]['box_A'].max()
# max_final_A
slow_runs = df[(df['timestep']==df['timestep'].max()) & 
         (df['box_A']==max_final_A)]['run']
slow_runs = list(slow_runs)
slow_runs

ax = None
for i in slow_runs:
    ax = df[df['run']==i].plot('timestep', ['box_A', 'box_B'],
                                     grid=True,
                                     xticks=list(df['timestep'].drop_duplicates()), 
                                     yticks=list(range(1+max(df['box_A'].max(),df['box_B'].max()))),
                                     legend = (ax == None),
                                     colormap = 'RdYlGn',
                                     ax = ax
                                  )

We invite the reader to fork this code and come up with answers for their other questions that might be interesting to look at. For example:
* How often does box B momentarily contain more marbles than box A?
* What's the frequency distribution of the time to reach equilibrium?
* What's the probability distribution of the waiting times of each one of the robots?